The following exercises use 2019 flight statistics from the United States Department of Transportation’s Bureau of Transportation Statistics (available [here](https://www.transtats.bts.gov/DL_SelectFields.asp?gnoyr_VQ=FMF&QO_fu146_anzr=Nv4%20Pn44vr45) and in this repository as `data/T100_MARKET_ALL_CARRIER.zip`). Each row contains information about a specific route for a given carrier in a given month (e.g., JFK &rarr; LAX on Delta Airlines in January). There are 321,409 rows and 41 columns. Note that you don't need to unzip the file to read it in with `pd.read_csv()`.

This continues last week's exercises.

#### __Exercise 8__:

Find the top 5 carriers for median route distance (i.e., for all origin-destination pairs that each carrier has, find the median distance after removing duplicates).

In [21]:
import pandas as pd

data = pd.read_csv('./data/T100_MARKET_ALL_CARRIER.zip')


df_unique = data.drop_duplicates(subset=['UNIQUE_CARRIER_NAME', 'ORIGIN', 'DISTANCE','DEST'])
median_distances = df_unique.groupby('UNIQUE_CARRIER_NAME')['DISTANCE'].median().nlargest(5)

print(median_distances)

UNIQUE_CARRIER_NAME
Singapore Airlines Ltd.                                      8,068.00
Cathay Pacific Airways Ltd.                                  8,020.00
Qantas Airways Ltd.                                          7,886.00
Longtail Aviation Ltd.                                       7,855.50
National Aviation Company of India Limited d/b/a Air India   7,798.00
Name: DISTANCE, dtype: float64


#### __Exercise 9:__ 
Find the total cargo transported (mail + freight) and mean distance traveled for the 10 carriers that transported the most cargo.

In [29]:
pd.set_option('display.float_format', '{:,.2f}'.format)

data['TOTAL_CARGO'] = data['MAIL']+data['FREIGHT']

top_10_carriers = data.groupby('CARRIER')['TOTAL_CARGO'].sum().nlargest(10).index

top_10_data = data[data['CARRIER'].isin(top_10_carriers)]

result = top_10_data.groupby('CARRIER').agg({'TOTAL_CARGO': 'sum', 'DISTANCE': 'mean'}).reset_index()

print(result)

  CARRIER       TOTAL_CARGO  DISTANCE
0      5X  9,173,866,790.00  1,030.81
1      5Y  3,356,847,309.00  1,900.99
2      AA  1,353,074,248.00  1,583.19
3      CI    837,079,895.00  5,828.28
4      CX    774,177,697.00  7,498.69
5      DL  1,129,523,987.00  1,612.69
6      FX 12,709,663,842.00  1,121.89
7     KAQ  1,272,180,283.00  2,519.58
8      PO  1,199,386,139.00  3,111.79
9      UA  1,577,891,983.00  1,810.66


In [27]:
data.assign(cargo=lambda x: x.MAIL + x.FREIGHT)\
    .groupby('UNIQUE_CARRIER_NAME')[['cargo','DISTANCE']]\
    .agg(dict(cargo='sum', DISTANCE='mean')).nlargest(10,'cargo')

,cargo,DISTANCE
UNIQUE_CARRIER_NAME,,
Federal Express Corporation,"12,709,663,842.00","1,121.89"
United Parcel Service,"9,173,866,790.00","1,030.81"
Atlas Air Inc.,"3,356,847,309.00","1,900.99"
United Air Lines Inc.,"1,577,891,983.00","1,810.66"
American Airlines Inc.,"1,353,074,248.00","1,583.19"
Kalitta Air LLC,"1,272,180,283.00","2,519.58"
Polar Air Cargo Airways,"1,199,386,139.00","3,111.79"
Delta Air Lines Inc.,"1,129,523,987.00","1,612.69"
China Airlines Ltd.,"837,079,895.00","5,828.28"


#### __Exercise 10__:

Which 10 carriers flew the most passengers out of the United States to another country?

In [18]:
us_to_international = data[(data['ORIGIN_COUNTRY'] == 'US') & (data['DEST_COUNTRY'] != 'US')]
top_10_carriers_passengers = us_to_international.groupby('CARRIER')['PASSENGERS'].sum().nlargest(10).reset_index()


print(top_10_carriers_passengers)


  CARRIER    PASSENGERS
0      AA 14,867,653.00
1      UA 14,427,923.00
2      DL 13,054,230.00
3      B6  4,522,492.00
4      BA  3,758,945.00
5      LH  3,123,611.00
6      WS  2,626,600.00
7      AC  2,540,855.00
8      WN  2,146,960.00
9      VS  2,074,735.00


In [32]:
top_10_passengers = data\
    .query('ORIGIN_COUNTRY_NAME == "United States" and DEST_COUNTRY_NAME != "United States"')\
    .groupby('UNIQUE_CARRIER_NAME')['PASSENGERS'].sum().nlargest(10)

print(top_10_passengers)

UNIQUE_CARRIER_NAME
American Airlines Inc.      14,867,653.00
United Air Lines Inc.       14,427,923.00
Delta Air Lines Inc.        13,054,230.00
JetBlue Airways              4,522,492.00
British Airways Plc          3,758,945.00
Lufthansa German Airlines    3,123,611.00
Westjet                      2,626,600.00
Air Canada                   2,540,855.00
Southwest Airlines Co.       2,146,960.00
Virgin Atlantic Airways      2,074,735.00
Name: PASSENGERS, dtype: float64
